# NOTAS
##### A VERSAO COMMUNITY NAO PERMITINDO CRIAR CLUSTERS 
##### UTILIZER O SQL WEREHOUSE
##### O SQL WEREHOUSE TEM UMA LIMITACAO EM RELACAO A REQUISICAO ULTILIZANDO ALGUMAS LIBS DO PYHTON (RESQUES POR EXEMPLO)
##### CRIEI A FUNCAO EM UM NOTEBOOK PYTHON EXTERNO E GEREI OS DADOS DA TABELA E FORMATO CSV E CARREGEI NO WORKSPACE

## LIBS EXTRAS

In [0]:
!pip install bs4

## BIBLIOTECAS

In [0]:
from bs4 import BeautifulSoup

import requests

## LEITURA DA TABEALA DE APOIO

In [0]:
df_creators = spark.table('default.creators_scrape_wiki')

wiki_pages = [row.wiki_page for row in df_creators.collect()]

## FUNCAO DE COLETA ID_USER

In [0]:
def get_id_user_youtube(page_name):

    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "parse",
        "page": page_name,
        "format": "json"
    }
    try:
        r = requests.get(url, params=params, timeout=10)
        r.raise_for_status()
        data = r.json()

        html = data.get("parse", {}).get("text", {}).get("*", "")
        soup = BeautifulSoup(html, "html.parser")

        for a in soup.find_all("a", href=True):
            href = a["href"]
            if "youtube.com" in href:
                if "/user/" in href:
                    return href.split("/user/")[-1].split("/")[0]
                elif "/channel/" in href:
                    return href.split("/channel/")[-1].split("/")[0]
        return None
    except Exception as e:
        print(f"ERRO - {page_name}: {e}")
        return None

## LOOP NA API

In [0]:
dados = []
for page in wiki_pages:
    user_id = get_id_user_youtube(page)
    if user_id:
        dados.append((user_id, page))

df_users_yt = spark.createDataFrame(dados, ["user_id", "wiki_page"])

## GRAVAR TABELA

In [0]:
df_users_yt.write\
           .format('delta')\
           .mode('overwrite')\
           .option("overwriteSchema", "true")\
           .saveAsTable("default.users_yt")